In [1]:
import h5py
import numpy as np
from keras.models import Sequential, load_model, Model
import pickle
from keras.layers import Dense,Flatten,Conv2D,MaxPool2D
from keras.optimizers import SGD
from keras.utils import to_categorical
import cv2
import os
import json
import sys

Using TensorFlow backend.


In [2]:
model = load_model('data/lenet5_relu_model.h5')

In [3]:
model_hdf5 = h5py.File("data/lenet5_relu_model.h5")

In [4]:
# 获取模型的所有参数
model_config = model.get_config()

In [5]:
json_string = model.to_json()

In [6]:
imgPath = '/home/szh-920/workspace/master_graduate/cnn_code/first_train_model/data/GTSRB_train_jpgs/00012#00037_00019.jpg'
picture = cv2.imread(imgPath)

In [7]:
picture.shape

(72, 72, 3)

In [8]:
picture.dtype

dtype('uint8')

In [9]:
cv2.imwrite("origin.jpg", picture)
DST_SIZE = (35, 35, 1)

In [10]:
mat = cv2.resize(cv2.cvtColor(picture, cv2.COLOR_BGR2GRAY), (DST_SIZE[0], DST_SIZE[1]))
cv2.imwrite("gray1.jpg", mat)


True

In [11]:
mat2 = mat.reshape(DST_SIZE)

In [12]:
mat.shape

(35, 35)

In [13]:
mat2.shape

(35, 35, 1)

In [14]:
origin_gray_picture = mat.tolist()
model_input_mat = mat2.tolist()

In [15]:
conv2d_1_layer = model.get_layer("conv2d_1")

In [16]:
model_all_weights = model.get_weights()

In [17]:
conv2d_1_params = conv2d_1_layer.get_weights()

In [18]:
#params[0] is kernal
conv2d_1_params[0].shape

(5, 5, 1, 6)

In [19]:
#params[1] is bias
conv2d_1_params[0]

array([[[[-8.05068538e-02, -3.66332405e-03,  3.19653600e-02,
          -1.33803397e-01, -1.41846538e-01, -2.14381739e-01]],

        [[-8.17928929e-03, -5.46407819e-01, -5.55108547e-01,
          -1.53266370e-01,  8.07685312e-03, -2.78513193e-01]],

        [[-1.56415194e-01, -1.26194406e+00, -6.04668438e-01,
          -1.35470647e-02, -8.04923326e-02,  1.02636859e-01]],

        [[ 1.69596747e-01, -1.09678829e+00, -4.93592650e-01,
           3.30345668e-02,  7.77080283e-02, -7.34073594e-02]],

        [[ 3.65295447e-02, -5.46850383e-01, -1.17155552e-01,
           4.06704582e-02, -1.55465439e-01, -2.33241454e-01]]],


       [[[-1.87954977e-01,  2.59794086e-01, -2.58282334e-01,
          -1.29971012e-01, -1.29813254e-01, -2.82143891e-01]],

        [[ 1.35280967e-01,  1.31916538e-01, -1.00204277e+00,
          -1.85269594e-01, -6.58444464e-02, -1.32326946e-01]],

        [[-1.16012320e-01, -4.24970716e-01, -7.24410355e-01,
          -9.82290581e-02,  4.67756726e-02,  2.25507617e-01]],

In [20]:
f = open("data/jpgs_np_arrays_dataset.pickle", "rb")
dataSet = pickle.load(f)
f.close()

In [21]:
dataSet.keys()

dict_keys(['train_x', 'train_y', 'test_x', 'test_y'])

In [22]:
train_x = dataSet['train_x']

In [23]:
res = model.predict(train_x)

In [24]:
res.shape

(35288, 43)

In [25]:
res[0]

array([3.38258981e-16, 2.28342389e-09, 5.90623640e-05, 1.82762979e-05,
       3.52011135e-18, 9.99922633e-01, 6.33534540e-08, 1.51705350e-11,
       2.74153847e-26, 7.53928419e-26, 6.43076413e-17, 5.51545486e-20,
       2.23391676e-21, 1.89024747e-27, 6.18893802e-24, 1.50086164e-22,
       4.65136463e-17, 2.43096746e-30, 8.89364943e-27, 1.38985252e-19,
       3.08765262e-20, 4.90130817e-17, 2.03555252e-25, 1.22610149e-18,
       8.54591194e-23, 7.73951592e-24, 1.02260891e-22, 7.81404809e-21,
       1.08572023e-23, 3.34562837e-25, 7.01467486e-23, 1.33856369e-17,
       3.31724043e-16, 3.74772655e-25, 1.12479998e-24, 1.25688912e-24,
       3.37283050e-19, 1.04148663e-21, 8.53717578e-24, 1.17899680e-23,
       4.87742222e-22, 1.74672612e-20, 1.32173765e-14], dtype=float32)

In [26]:
test_y = dataSet['test_y']
test_x = dataSet['test_x']
train_x = dataSet['train_x']
train_y = dataSet['train_y']

In [27]:
test_y.shape

(3921, 43)

In [28]:
test_y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [29]:
def max_idx(li):
    max_idx = 0
    max_val = -100000.0
    for idx, val in enumerate(li):
        if val > max_val:
            max_val = val
            max_idx = idx
    return idx
pred_test_y = model.predict(train_x).tolist()
test_y = train_y.tolist()
cnt = 0
for i in range(35288):
    a = pred_test_y[i]
    b = test_y[i]
    if max_idx(a) == max_idx(b):
        cnt += 1
print(cnt)

35288


In [36]:
test_imgs = test_x[0:1]
test_imgs.shape

(1, 35, 35, 1)

In [37]:
conv2d_1_layer_model = Model(input=model.input, output=conv2d_1_layer.output)
conv2d_1_layer_model_output = conv2d_1_layer_model.predict(test_imgs)

/home/szh-920/apps/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("co..., outputs=Tensor("co...)`
  if __name__ == '__main__':


In [38]:
conv2d_1_layer_model_output.shape

(1, 31, 31, 6)

In [39]:
conv2d_1_layer_model_output

array([[[[0.        , 0.4221774 , 0.32896608, 0.        , 0.        ,
          0.        ],
         [0.        , 0.5275756 , 0.29752177, 0.        , 0.        ,
          0.        ],
         [0.        , 0.6283324 , 0.38644457, 0.        , 0.        ,
          0.        ],
         ...,
         [0.        , 0.546126  , 0.6532588 , 0.        , 0.        ,
          0.05077969],
         [0.        , 0.47734123, 0.5954661 , 0.        , 0.        ,
          0.0590717 ],
         [0.        , 0.48888618, 0.5494079 , 0.        , 0.        ,
          0.04607467]],

        [[0.        , 0.5615872 , 0.45301476, 0.        , 0.        ,
          0.        ],
         [0.        , 0.68484414, 0.44017258, 0.        , 0.        ,
          0.        ],
         [0.        , 0.67657185, 0.42990944, 0.        , 0.        ,
          0.        ],
         ...,
         [0.        , 0.6250146 , 0.60711503, 0.        , 0.        ,
          0.        ],
         [0.        , 0.5072088 , 0.4960